In [1]:
import os
from pelops.datasets.chipper import FrameProducer, ExtractedChip, Methods, Chipper
import cv2
from hdfs3 import HDFileSystem

hdfs = HDFileSystem(host='namenode', port=8020)
filenames = hdfs.glob('/datasets/texas_dot/Ending20160903/user/*.mp4')

def get_info(filename):
    bname = os.path.basename(filename)
    return bname.split('-')[0], int(bname.split('-')[1])

camera_name = 'IH10_Martin_20160903T225628-0'
camera_name = 'IH37_Jones'

filenames = sorted(filenames, key=get_info)
filenames_filtered = [filename for filename in filenames if camera_name in filename]

import glob
filenames = sorted(glob.glob('/local_data/yonas/tmp/LA/2017/072/Claiborne1*.mp4'))[:-1][:5]
fp = FrameProducer([filenames[0]], open_func=str, desired_framerate=5)

#fp = FrameProducer([filenames_filtered[0]], hdfs.open)

def apply_mask(input_arry):
    input_arry[:125] = 0
    input_arry[210:] = 0
    return input_arry

def expand_box(x,y,w,h, amount=5):
    return x-amount, y-amount, w+amount*2, h+amount*2

In [2]:
def get_frame_lookup(fp, 
                     kernel_size=(25,25), 
                     threshold=30, 
                     chipping_method=Methods.BACKGROUND_SUB
                    ):
    chipper = Chipper(fp, 
                  mask_modifier=None,#mask_modifier,
                  kernel_size = kernel_size,
                  threshold = threshold,
                  box_expander=None,#expand_box,
                  chipping_method=chipping_method)

    # Get chips by frame number
    chips = []
    for frame_chips in chipper:
        chips.extend(frame_chips)

    from collections import defaultdict
    frame_lookup = defaultdict(list)
    for ec in chips:
        frame_lookup[ec.frame_number].append(ec)
    return frame_lookup, chips

In [3]:
import xml.etree.ElementTree as ET
from collections import namedtuple
def get_xml_name(filename, frame_number):
    # Get XML file name based on orignal image file name and frame
    return '{}_{}_.xml'.format(os.path.basename(filename),frame_number)

BBox =namedtuple('BBox', ['x','y', 'w', 'h'])

def get_x_y_w_h(bbox):
    xs = [x for x,y in bbox]
    ys = [y for x,y in bbox]
    min_x = min(xs)
    min_y = min(ys)
    max_x = max(xs)
    max_y = max(ys)
    return BBox(x=min_x, 
                y=min_y, 
                w=max_x-min_x, 
                h=max_y-min_y)

def get_bboxes_from_xml(xml_obj):
    for top_level_tag in xml_obj.iter('filename'):
        frame_filename = top_level_tag.text
    bboxes = []
    for top_level_tag in xml_obj.iter('object'):
        for polygon in top_level_tag.iter('polygon'):
            pts = []
            for pt in polygon.iter('pt'):
                x = None
                y = None
                for dim in pt.iter():
                    if dim.tag == 'x':
                        x = int(dim.text)
                    elif dim.tag == 'y':
                        y = int(dim.text)
                if x and y:
                    pts.append((x,y))
            bboxes.append(get_x_y_w_h(pts))
    return frame_filename, bboxes

def get_bboxes_from_file(filename):
    tree = ET.parse(filename)
    root = tree.getroot()
    return get_bboxes_from_xml(root)

In [4]:
import glob
xml_basepath = '/data/fs4/teams/pelops/labelme/annotations/'
xml_files_lookup = {}
for filename in glob.glob(os.path.join(xml_basepath, '*')):
    xml_files_lookup[os.path.basename(filename)] = filename

In [5]:
def get_intersection_union(bbox1, bbox2):
    # Top left of intersection
    xA = max(bbox1.x, bbox2.x)
    yA = max(bbox1.y, bbox2.y)
    
    # Bottom right
    xB = min(bbox1.x+bbox1.w, bbox2.x+bbox2.w)
    yB = min(bbox1.y+bbox1.h, bbox2.y+bbox2.h)
    width = xB-xA
    height = yB-yA
    if width < 0 or height < 0:
        return None
    return (xA, yA, width, height)

#def get_iou(bbox1, bbox2):
#    _, _, w, h = get_intersection(bbox1, bbox2)
#    return 

def get_iou(bbox1, bbox2):
    # Top left of intersection
    xA = max(bbox1.x, bbox2.x)
    yA = max(bbox1.y, bbox2.y)
    
    # Bottom right
    xB = min(bbox1.x+bbox1.w, bbox2.x+bbox2.w)
    yB = min(bbox1.y+bbox1.h, bbox2.y+bbox2.h)
    width = xB-xA
    height = yB-yA
    if width < 0 or height < 0:
        return 0
    else:
        intersectionArea = (xB - xA + 1) * (yB - yA + 1)
        unionArea = bbox1.w*bbox1.h + bbox2.w*bbox2.h
        
        return intersectionArea/float(unionArea - intersectionArea)

    return (xA, yA, width, height)   

In [6]:
%matplotlib inline
import matplotlib.pyplot as plt

In [7]:
from PIL import Image
import numpy as np
def visualize_perf(filename, bboxes1, bboxes2):
    frame = Image.open('/data/fs4/teams/pelops/labelme/images/{}'.format(filename))
    frame = np.uint8(frame)
    for box in bboxes1:
        x, y, w, h = box.x, box.y, box.w, box.h
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),1)
    for box in bboxes2:
        x, y, w, h = box.x, box.y, box.w, box.h
        cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),1)
    
    intersection = get_intersection_union(bboxes1[0], bboxes2[0])
    if intersection:
        x,y,w,h = intersection
        cv2.rectangle(frame,(x,y),(x+w,y+h),(0,0,255),1)
    plt.imshow(frame)

In [8]:
import itertools
def score_frame_lookup(frame_lookup, chips):
    video_ious = []
    for frame_number in sorted(frame_lookup):
        chips = frame_lookup[frame_number]
        bname = os.path.basename(chips[0].filename)
        xml_filename = get_xml_name(bname, frame_number)
        if xml_filename in xml_files_lookup:
            xml_full_filename = xml_files_lookup[xml_filename]
            frame_image, truth_bboxes = get_bboxes_from_file(xml_full_filename)
            found_bboxes = []
            for chip in chips:
                found_bboxes.append((chip.x, chip.y, chip.w, chip.h))
            frame_ious = []
            for truth_bbox in truth_bboxes:
                box_ious = []
                for chip in chips:
                    box_ious.append(get_iou(chip, truth_bbox))
                frame_ious.append(max(box_ious))
            video_ious.append(np.mean(frame_ious))
            #visualize_perf(frame_image, truth_bboxes, chips)
            #break
    return np.mean(video_ious)

In [41]:
frame_number = 3000
chips = frame_lookup[frame_number]
bname = os.path.basename(chips[0].filename)
xml_filename = get_xml_name(bname, frame_number)

In [42]:
xml_filename in xml_files_lookup

True

In [25]:
get_xml_name(os.path.basename(frame_lookup[14340][0].filename), 14340) in xml_files_lookup

False

In [ ]:
kernel_size = (25,25)
threshold = 30
chipping_method = Methods.BACKGROUND_SUB
#chipping_method=Methods.OPENCV 
mask_modifier= None # apply_mask
score_strings = []
for kernel_size in [(5,5), (7,7), (9,9), (33,33)]:
    for threshold in [5, 10,12, 15, 20, 30, 50]:
        for chipping_method in [Methods.BACKGROUND_SUB, Methods.OPENCV]:
            frame_lookup, chips = get_frame_lookup(fp, 
                                                   kernel_size=kernel_size,
                            threshold=threshold, 
                            chipping_method=chipping_method)
            score = score_frame_lookup(frame_lookup, chips)
            print('{}, {}, {}: {}'.format(kernel_size, threshold, chipping_method, score))
            score_strings.append('{}, {}, {}: {}'.format(kernel_size, threshold, chipping_method, score))

(5, 5), 5, Methods.BACKGROUND_SUB: 0.20377665195653025
(5, 5), 5, Methods.OPENCV: 0.6079182954829871
(5, 5), 10, Methods.BACKGROUND_SUB: 0.4296029757968663
(5, 5), 10, Methods.OPENCV: 0.584373918897133
(5, 5), 12, Methods.BACKGROUND_SUB: 0.47808368950878416
(5, 5), 12, Methods.OPENCV: 0.5807517854640142
(5, 5), 15, Methods.BACKGROUND_SUB: 0.5357553431412312
(5, 5), 15, Methods.OPENCV: 0.5679602734960262
(5, 5), 20, Methods.BACKGROUND_SUB: 0.6118601486912363
(5, 5), 20, Methods.OPENCV: 0.5479004331432262
(5, 5), 30, Methods.BACKGROUND_SUB: 0.6545387798259704
(5, 5), 30, Methods.OPENCV: 0.5129981766403243
(5, 5), 50, Methods.BACKGROUND_SUB: 0.5909724610090009
(5, 5), 50, Methods.OPENCV: 0.4468784050260238


In [14]:
for m in Methods:
    print(m)

Methods.OPENCV
Methods.BACKGROUND_SUB


In [15]:
for method in Methods:
    for line in score_strings:
        if str(method) in line:
            print(line)

(5, 5), 5, Methods.OPENCV: 0.6079182954829871
(5, 5), 10, Methods.OPENCV: 0.584373918897133
(5, 5), 12, Methods.OPENCV: 0.5807517854640142
(5, 5), 15, Methods.OPENCV: 0.5679602734960262
(5, 5), 20, Methods.OPENCV: 0.5479004331432262
(5, 5), 30, Methods.OPENCV: 0.5129981766403243
(5, 5), 50, Methods.OPENCV: 0.4468784050260238
(7, 7), 5, Methods.OPENCV: 0.6383763857018694
(7, 7), 10, Methods.OPENCV: 0.5985662893550552
(7, 7), 12, Methods.OPENCV: 0.5885574381525973
(7, 7), 15, Methods.OPENCV: 0.574304704233907
(7, 7), 20, Methods.OPENCV: 0.5600960235270905
(7, 7), 30, Methods.OPENCV: 0.5109526569511246
(7, 7), 50, Methods.OPENCV: 0.44988477427898066
(9, 9), 5, Methods.OPENCV: 0.6485522067967696
(9, 9), 10, Methods.OPENCV: 0.6157400687894344
(9, 9), 12, Methods.OPENCV: 0.5995303234331684
(9, 9), 15, Methods.OPENCV: 0.5844991760180644
(9, 9), 20, Methods.OPENCV: 0.5644634882621449
(9, 9), 30, Methods.OPENCV: 0.5220527447211238
(9, 9), 50, Methods.OPENCV: 0.4513636869249082
(33, 33), 5, Meth

In [ ]:
p